In [1]:
#Required libraries for the program

import pandas as pd
import json
import os
import git
import mysql.connector
from sqlalchemy import create_engine



In [2]:
#using this to avoid creating duplicate values inside tables
mydb = mysql.connector.connect(host="localhost",user="root",password="")

mycursor = mydb.cursor(buffered=True)

mycursor.execute('drop database if exists phonepe')

In [3]:
#created the clone of GIT-HUB repository then,


try:
    repository_url = "https://github.com/PhonePe/pulse.git"

    destination_directory = "D:/PythonProjects/PHONEPE"

    git.Repo.clone_from(repository_url, destination_directory)
except:
    print("already exists")


already exists


In [4]:
#STATE NAME CORRECTION  
#Agg_state_list--> to get the list of states in India

state_mapping = {
    'andaman-&-nicobar-islands': 'Andaman and Nicobar Islands',
    'tamil-nadu': 'Tamil Nadu',
    'lakshadweep': 'Lakshadweep',
    'telangana': 'Telangana',
    'manipur': 'Manipur',
    'haryana': 'Haryana',
    'gujarat': 'Gujarat',
    'sikkim': 'Sikkim',
    'delhi': 'Delhi',
    'west-bengal': 'West Bengal',
    'uttar-pradesh': 'Uttar Pradesh',
    'goa': 'Goa',
    'punjab': 'Punjab',
    'arunachal-pradesh': 'Arunachal Pradesh',
    'karnataka': 'Karnataka',
    'jammu-&-kashmir': 'Jammu and Kashmir',
    'maharashtra': 'Maharashtra',
    'odisha': 'Odisha',
    'madhya-pradesh': 'Madhya Pradesh',
    'rajasthan': 'Rajasthan',
    'andhra-pradesh': 'Andhra Pradesh',
    'chandigarh': 'Chandigarh',
    'kerala': 'Kerala',
    'chhattisgarh': 'Chhattisgarh',
    'tripura': 'Tripura',
    'mizoram': 'Mizoram',
    'himachal-pradesh': 'Himachal Pradesh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'ladakh': 'Ladakh',
    'assam': 'Assam',
    'meghalaya': 'Meghalaya',
    'uttarakhand': 'Uttarakhand',
    'puducherry': 'Puducherry',
    'bihar': 'Bihar',
    'jharkhand': 'Jharkhand',
    'nagaland': 'Nagaland'
}

In [5]:
#Aggregate transaction
#This is to direct the path to get the data as states

path="Data/data/aggregated/transaction/country/india/state/"
Agg_trans_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe

abc={'State':[], 'Year':[],'Quater':[],'Transaction_type':[], 'Transaction_count':[], 'Transaction_amount':[]}

for i in Agg_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        Agg_yr=os.listdir(p_i)
        for j in Agg_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                Agg_yr_list=os.listdir(p_j)
                for k in Agg_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D['data']['transactionData']:
                                Name=z['name']
                                count=z['paymentInstruments'][0]['count']
                                amount=z['paymentInstruments'][0]['amount']
                                abc['Transaction_type'].append(Name)
                                abc['Transaction_count'].append(count)
                                abc['Transaction_amount'].append(amount)
                                abc['State'].append(i)
                                abc['Year'].append(j)
                                abc['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_Trans=pd.DataFrame(abc)
Agg_Trans['State'] = Agg_Trans['State'].map(state_mapping)

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysqlalchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('create database if not exists phonepe')
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Agg_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                Transaction_type VARCHAR(40), 
                                                Transaction_count BIGINT, 
                                                Transaction_amount BIGINT )  ''')
mydb.commit()

#migrating to mysql
Agg_Trans.to_sql(name='Agg_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()



C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\1339012696.py:58: UserWarning: The provided table name 'Agg_transaction' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  Agg_Trans.to_sql(name='Agg_transaction', con=db_engine, if_exists='append', index=False)


In [6]:
Agg_Trans

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,Andaman and Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman and Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman and Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05
3,Andaman and Nicobar Islands,2018,1,Financial Services,33,1.060142e+04
4,Andaman and Nicobar Islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
4489,West Bengal,2024,1,Merchant payments,407607197,2.732137e+11
4490,West Bengal,2024,1,Peer-to-peer payments,355646156,1.100744e+12
4491,West Bengal,2024,1,Recharge & bill payments,72216060,4.716192e+10
4492,West Bengal,2024,1,Financial Services,451008,5.307383e+08


In [7]:
#Aggregate User
#This is to direct the path to get the data as states

path="Data/data/aggregated/user/country/india/state/"
Agg_user_state_list=os.listdir(path)

#Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

#This is to extract the data's to create a dataframe

abc2={'State':[], 'Year':[],'Quater':[],'Registered_PhonePe_users':[], 'PhonePe_app_opens':[]}

for i in Agg_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        Agg_yr=os.listdir(p_i)
        for j in Agg_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                Agg_yr_list=os.listdir(p_j)
                for k in Agg_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D:
                                users=D['data']['aggregated']['registeredUsers']
                                opens=D['data']['aggregated']['appOpens']
                                abc2['Registered_PhonePe_users'].append(users)
                                abc2['PhonePe_app_opens'].append(opens)
                                abc2['State'].append(i)
                                abc2['Year'].append(j)
                                abc2['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_user=pd.DataFrame(abc2)
Agg_user['State'] = Agg_user['State'].map(state_mapping)

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Agg_user(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                Registered_PhonePe_users BIGINT, 
                                                PhonePe_app_opens BIGINT)  ''')
mydb.commit()

#migrating to mysql
Agg_user.to_sql(name='Agg_user', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()



C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\627603452.py:58: UserWarning: The provided table name 'Agg_user' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  Agg_user.to_sql(name='Agg_user', con=db_engine, if_exists='append', index=False)


In [8]:
Agg_user

,State,Year,Quater,Registered_PhonePe_users,PhonePe_app_opens
0,Andaman and Nicobar Islands,2018,1,6740,0
1,Andaman and Nicobar Islands,2018,1,6740,0
2,Andaman and Nicobar Islands,2018,1,6740,0
3,Andaman and Nicobar Islands,2018,1,6740,0
4,Andaman and Nicobar Islands,2018,2,9405,0
...,...,...,...,...,...
3595,West Bengal,2023,4,30064546,1467442959
3596,West Bengal,2024,1,31306843,633526507
3597,West Bengal,2024,1,31306843,633526507
3598,West Bengal,2024,1,31306843,633526507


In [9]:
#Map transaction

#This is to direct the path to get the data as states

path="Data/data/map/transaction/hover/country/india/state/"
Map_trans_state_list=os.listdir(path)

#Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

#This is to extract the data's to create a dataframe

abc3={'State':[], 'Year':[],'Quater':[], 'District':[], 'All_Transactions':[], 'Total_payment_value':[]}

for i in Map_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        map_yr=os.listdir(p_i)
        for j in map_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                map_yr_list=os.listdir(p_j)
                for k in map_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            for z in D:
                                dist=D['data']['hoverDataList'][0]['name']
                                totaltrans=D['data']['hoverDataList'][0]['metric'][0]['count']
                                amounts=D['data']['hoverDataList'][0]['metric'][0]['amount']
                                abc3['All_Transactions'].append(totaltrans)
                                abc3['Total_payment_value'].append(amounts)
                                abc3['District'].append(dist)
                                abc3['State'].append(i)
                                abc3['Year'].append(j)
                                abc3['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
map_trans=pd.DataFrame(abc3)
map_trans['State'] = map_trans['State'].map(state_mapping)


#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Map_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(50),
                                                All_Transactions BIGINT, 
                                                Total_payment_value BIGINT )  ''')
mydb.commit()

#migrating to mysql
map_trans.to_sql(name='Map_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()




C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\1665393323.py:63: UserWarning: The provided table name 'Map_transaction' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  map_trans.to_sql(name='Map_transaction', con=db_engine, if_exists='append', index=False)


In [10]:
map_trans

,State,Year,Quater,District,All_Transactions,Total_payment_value
0,Andaman and Nicobar Islands,2018,1,north and middle andaman district,442,9.316631e+05
1,Andaman and Nicobar Islands,2018,1,north and middle andaman district,442,9.316631e+05
2,Andaman and Nicobar Islands,2018,1,north and middle andaman district,442,9.316631e+05
3,Andaman and Nicobar Islands,2018,1,north and middle andaman district,442,9.316631e+05
4,Andaman and Nicobar Islands,2018,2,north and middle andaman district,825,1.317863e+06
...,...,...,...,...,...,...
3595,West Bengal,2023,4,south twenty four parganas district,46005273,8.433607e+10
3596,West Bengal,2024,1,south twenty four parganas district,53004493,9.904291e+10
3597,West Bengal,2024,1,south twenty four parganas district,53004493,9.904291e+10
3598,West Bengal,2024,1,south twenty four parganas district,53004493,9.904291e+10


In [11]:

#Map user

#This is to direct the path to get the data as states

path="Data/data/map/user/hover/country/india/state/"
Map_user_state_list=os.listdir(path)
abc4={'State':[], 'Year':[],'Quater':[],'District':[],'Total_registered_users':[], 'Total_app_opens':[]}

for i in Map_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        map_yr=os.listdir(p_i)
        for j in map_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                map_yr_list=os.listdir(p_j)
                for k in map_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            key_list=list(D['data']['hoverData'].keys())
                            for dist in key_list:
                                abc4['State'].append(i)
                                abc4['Year'].append(j)
                                abc4['Quater'].append(int(k.strip('.json')))
                                abc4['District'].append(dist)
                                abc4['Total_registered_users'].append(D['data']['hoverData'][dist]['registeredUsers'])
                                abc4['Total_app_opens'].append(D['data']['hoverData'][dist]['appOpens'])

map_user=pd.DataFrame(abc4)
map_user['State'] = map_user['State'].map(state_mapping)

chunk_size=1000

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Map_user(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(100),
                                                Total_registered_users BIGINT, 
                                                Total_app_opens BIGINT )  ''')
mydb.commit()

#migrating to mysql
#map_user.to_sql(name='Map_user', con=db_engine, if_exists='append', index=False)

for i in range(0, len(map_user), chunk_size):
    chunk = map_user[i:i+chunk_size]
    chunk.to_sql(name='Map_user', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()


C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\23152544.py:59: UserWarning: The provided table name 'Map_user' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  chunk.to_sql(name='Map_user', con=db_engine, if_exists='append', index=False)


In [12]:
map_user

,State,Year,Quater,District,Total_registered_users,Total_app_opens
0,Andaman and Nicobar Islands,2018,1,north and middle andaman district,632,0
1,Andaman and Nicobar Islands,2018,1,south andaman district,5846,0
2,Andaman and Nicobar Islands,2018,1,nicobars district,262,0
3,Andaman and Nicobar Islands,2018,2,north and middle andaman district,911,0
4,Andaman and Nicobar Islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
18295,West Bengal,2024,1,nadia district,1671140,13679202
18296,West Bengal,2024,1,birbhum district,1056537,30502875
18297,West Bengal,2024,1,purba medinipur district,1655920,18688405
18298,West Bengal,2024,1,maldah district,1183956,33270738


In [13]:

#top transaction
#This is to direct the path to get the data as states

path="Data/data/top/transaction/country/india/state/"
top_trans_state_list=os.listdir(path)
abc5={'State':[], 'Year':[],'Quater':[], 'District':[],'Districts_total_count':[],'Districts_total_amount':[], 'Postalcodes':[],'Postalcodes_total_count':[],'Postalcodes_total_amount':[]}

for i in top_trans_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        top_yr=os.listdir(p_i)
        for j in top_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                top_yr_list=os.listdir(p_j)
                for k in top_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            if D is not None and 'data' in D and 'states' in D['data'] and 'districts' in D['data'] and 'pincodes' in D['data']:
                                distname = D['data']['districts'][0]['entityName'] if D['data']['districts'] else "Unknown District"
                                postcode = D['data']['pincodes'][0]['entityName'] if D['data']['pincodes'] else "Unknown Postalcode"
                                distcount=D['data']['districts'][0]['metric']['count'] if D['data']['districts'] else "Unknown District count"
                                distamount=D['data']['districts'][0]['metric']['amount'] if D['data']['districts'] else "Unknown District amount"
                                postcodecount=D['data']['pincodes'][0]['metric']['count'] if D['data']['pincodes'] else "Unknown Postalcode count"
                                postcodeamount=D['data']['pincodes'][0]['metric']['amount'] if D['data']['pincodes'] else "Unknown Postalcode amount"
                                abc5['District'].append(distname)
                                abc5['Districts_total_count'].append(distcount)
                                abc5['Districts_total_amount'].append(distamount)
                                abc5['Postalcodes'].append(postcode)
                                abc5['Postalcodes_total_count'].append(postcodecount)
                                abc5['Postalcodes_total_amount'].append(postcodeamount)
                                abc5['State'].append(i)
                                abc5['Year'].append(j)
                                abc5['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
top_trans=pd.DataFrame(abc5)
top_trans['State'] = top_trans['State'].map(state_mapping)


chunk_size=1000

#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Top_transaction(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                District VARCHAR(100),
                                                Districts_total_count BIGINT, 
                                                Districts_total_amount BIGINT,
                                                Postalcodes INT(6),
                                                Postalcodes_total_count BIGINT,
                                                Postalcodes_total_amount BIGINT
                                                 )  ''')
mydb.commit()

#migrating to mysql

for i in range(0, len(top_trans), chunk_size):
    chunk = top_trans[i:i+chunk_size]
    chunk.to_sql(name='Top_transaction', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()


C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\118387127.py:70: UserWarning: The provided table name 'Top_transaction' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  chunk.to_sql(name='Top_transaction', con=db_engine, if_exists='append', index=False)


In [14]:
top_trans

,State,Year,Quater,District,Districts_total_count,Districts_total_amount,Postalcodes,Postalcodes_total_count,Postalcodes_total_amount
0,Andaman and Nicobar Islands,2018,1,south andaman,5688,1.256025e+07,744101,1622,2.769298e+06
1,Andaman and Nicobar Islands,2018,2,south andaman,9395,2.394824e+07,744101,2701,6.310090e+06
2,Andaman and Nicobar Islands,2018,3,south andaman,13511,4.426811e+07,744101,3462,1.083389e+07
3,Andaman and Nicobar Islands,2018,4,south andaman,20136,7.541184e+07,744101,4674,1.639286e+07
4,Andaman and Nicobar Islands,2019,1,south andaman,25683,8.550197e+07,744103,5742,1.940196e+07
...,...,...,...,...,...,...,...,...,...
895,West Bengal,2023,1,north twenty four parganas,82336235,1.343910e+11,700156,5958877,5.609483e+09
896,West Bengal,2023,2,north twenty four parganas,96628528,1.515717e+11,700156,6927595,6.467350e+09
897,West Bengal,2023,3,north twenty four parganas,113910456,1.668446e+11,700156,8289388,7.339618e+09
898,West Bengal,2023,4,north twenty four parganas,128343164,1.864662e+11,700156,8786298,7.815869e+09


In [15]:

#top user
#This is to direct the path to get the data as states

path="Data/data/top/user/country/india/state/"
top_user_state_list=os.listdir(path)
abc6={'State':[], 'Year':[],'Quater':[],'State_total_registered_user':[], 'District':[],'Districts_registered_user':[], 'Postalcodes':[],'Postalcodes_registered_user':[]}

for i in top_user_state_list:
    p_i=path+i+"/"
    if os.path.isdir(p_i):  # Check if it's a directory
        top_yr=os.listdir(p_i)
        for j in top_yr:
            p_j=p_i+j+"/"
            if os.path.isdir(p_j):  # Check if it's a directory
                top_yr_list=os.listdir(p_j)
                for k in top_yr_list:
                    p_k=p_j+k
                    if k.endswith('.json'):  # Check if it's a JSON file
                        with open(p_k, 'r') as Data:
                            D=json.load(Data)
                            if D is not None and 'data' in D and 'states' in D['data'] and 'districts' in D['data'] and 'pincodes' in D['data']:
                                distname = D['data']['districts'][0]['name'] if D['data']['districts'] else "Unknown District"
                                postcode = D['data']['pincodes'][0]['name'] if D['data']['pincodes'] else "Unknown Postalcode"
                                distuser=D['data']['districts'][0]['registeredUsers'] if D['data']['districts'] else "Unknown District count"
                                postcodeuser=D['data']['pincodes'][0]['registeredUsers'] if D['data']['pincodes'] else "Unknown Postalcode count"
                                stateuser=D['data']['states'][0]['registeredUsers'] if D['data']['states'] else "Unknown Postalcode amount"
                                abc6['State_total_registered_user'].append(distcount)
                                abc6['District'].append(distname)
                                abc6['Districts_registered_user'].append(distamount)
                                abc6['Postalcodes'].append(postcode)
                                abc6['Postalcodes_registered_user'].append(postcodecount)
                                abc6['State'].append(i)
                                abc6['Year'].append(j)
                                abc6['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
top_user=pd.DataFrame(abc6)
top_user['State'] = top_user['State'].map(state_mapping)

chunk_size=1000
#connectiong to mysql server
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)

#connecting to mysql alchemy
db_connection_str = f"mysql+mysqlconnector://root@localhost/phonepe"
db_engine = create_engine(db_connection_str)

#creating database and creating table
mycursor.execute('USE phonepe')
mycursor.execute('''create table if not exists  Top_users(State VARCHAR(50), 
                                                Year INT(4),
                                                Quater INT(2), 
                                                State_total_registered_user BIGINT, 
                                                District VARCHAR(100),
                                                Districts_registered_user BIGINT,
                                                Postalcodes BIGINT,
                                                Postalcodes_registered_user BIGINT
                                                )  ''')
mydb.commit()

#migrating to mysql
for i in range(0, len(top_user), chunk_size):
    chunk = top_user[i:i+chunk_size]
    chunk.to_sql(name='Top_users', con=db_engine, if_exists='append', index=False)

mycursor.close()
mydb.close()

C:\Users\RIYASPAUL\AppData\Local\Temp\ipykernel_25240\2826566520.py:64: UserWarning: The provided table name 'Top_users' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  chunk.to_sql(name='Top_users', con=db_engine, if_exists='append', index=False)


In [16]:
top_user

,State,Year,Quater,State_total_registered_user,District,Districts_registered_user,Postalcodes,Postalcodes_registered_user
0,Andaman and Nicobar Islands,2018,1,139181072,south andaman,2.030766e+11,744103,9224512
1,Andaman and Nicobar Islands,2018,2,139181072,south andaman,2.030766e+11,744103,9224512
2,Andaman and Nicobar Islands,2018,3,139181072,south andaman,2.030766e+11,744103,9224512
3,Andaman and Nicobar Islands,2018,4,139181072,south andaman,2.030766e+11,744103,9224512
4,Andaman and Nicobar Islands,2019,1,139181072,south andaman,2.030766e+11,744103,9224512
...,...,...,...,...,...,...,...,...
895,West Bengal,2023,1,139181072,north twenty four parganas,2.030766e+11,711101,9224512
896,West Bengal,2023,2,139181072,north twenty four parganas,2.030766e+11,711101,9224512
897,West Bengal,2023,3,139181072,north twenty four parganas,2.030766e+11,711101,9224512
898,West Bengal,2023,4,139181072,north twenty four parganas,2.030766e+11,711101,9224512


In [17]:
#dataframe creation from sql
#connection to sql
mydb = mysql.connector.connect(host="localhost",user="root",password="",database="phonepe")
mycursor = mydb.cursor(buffered=True)

#agg_transaction_df
mycursor.execute('select * from Agg_transaction')
mydb.commit()
df1=mycursor.fetchall()
Aggregate_transaction = pd.DataFrame(df1,columns=('State', 'Year','Quater','Transaction_type', 'Transaction_count', 'Transaction_amount'))


#agg_user_df
mycursor.execute('select * from Agg_user')
mydb.commit()
df2=mycursor.fetchall()
Aggregate_user = pd.DataFrame(df2,columns=('State', 'Year','Quater','Registered_PhonePe_users', 'PhonePe_app_opens'))

#Map_transaction_df
mycursor.execute('select * from Map_transaction')
mydb.commit()
df3=mycursor.fetchall()
Map_transactions = pd.DataFrame(df3,columns=('State', 'Year','Quater', 'District', 'All_Transactions', 'Total_payment_value'))

#map_user_df
mycursor.execute('select * from Map_user')
mydb.commit()
df4=mycursor.fetchall()
Map_users = pd.DataFrame(df4,columns=('State', 'Year','Quater','District','Total_registered_users', 'Total_app_opens'))

#Top_transaction_df
mycursor.execute('select * from Top_transaction')
mydb.commit()
df5=mycursor.fetchall()
Top_transactions = pd.DataFrame(df5,columns=('State', 'Year','Quater', 'District','Districts_total_count','Districts_total_amount', 'Postalcodes','Postalcodes_total_count','Postalcodes_total_amount'))

#Top_users_df
mycursor.execute('select * from Top_users')
mydb.commit()
df6=mycursor.fetchall()
Top_users_ = pd.DataFrame(df6,columns=('State', 'Year','Quater','State_total_registered_user', 'District','Districts_registered_user', 'Postalcodes','Postalcodes_registered_user'))


In [18]:
Aggregate_transaction
Aggregate_user

Map_transactions
Map_users

Top_transactions
Top_users_

,State,Year,Quater,State_total_registered_user,District,Districts_registered_user,Postalcodes,Postalcodes_registered_user
0,Andaman and Nicobar Islands,2018,1,139181072,south andaman,203076605812,744103,9224512
1,Andaman and Nicobar Islands,2018,2,139181072,south andaman,203076605812,744103,9224512
2,Andaman and Nicobar Islands,2018,3,139181072,south andaman,203076605812,744103,9224512
3,Andaman and Nicobar Islands,2018,4,139181072,south andaman,203076605812,744103,9224512
4,Andaman and Nicobar Islands,2019,1,139181072,south andaman,203076605812,744103,9224512
...,...,...,...,...,...,...,...,...
895,West Bengal,2023,1,139181072,north twenty four parganas,203076605812,711101,9224512
896,West Bengal,2023,2,139181072,north twenty four parganas,203076605812,711101,9224512
897,West Bengal,2023,3,139181072,north twenty four parganas,203076605812,711101,9224512
898,West Bengal,2023,4,139181072,north twenty four parganas,203076605812,711101,9224512
